# Urban Heat Island Analysis

#### Importing libraries and setting column width

In [43]:
import os
import json
import requests
import pandas as pd
from langchain_openai import ChatOpenAI #, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from pydantic import BaseModel, Field, validator, create_model
from typing import List, Optional, Text, Dict
from pprint import pprint
from IPython.display import display, Markdown
import plotly.express as px
import plotly.graph_objects as go
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, box
import fiona
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
pd.set_option('display.max_colwidth', 250)

### Grabbing Open AI key

In [45]:
# our OpenAI token
creds_file_open_ai = r"c:/Users/Chris/.secret/open_ai.txt"
with open(creds_file_open_ai) as f:
    creds_open_ai = f.read()

os.environ['OPENAI_API_KEY'] = creds_open_ai

You can access MODIS LST data through various APIs provided by NASA and other institutions. One of the most commonly used APIs is the **NASA Earthdata Search API**. Additionally, you can use the `pyproj` and `sentinelsat` libraries to access and download MODIS data.

### Accessing MODIS LST Data via NASA Earthdata API

To use the NASA Earthdata API, you will need to:

1. **Register for a NASA Earthdata Account:**
   - You need an Earthdata account to access the data. You can register at [Earthdata Login](https://urs.earthdata.nasa.gov/users/new).

2. **Use `pyproj` for Coordinate Transformations:**
   - If needed, you can use `pyproj` to handle coordinate transformations.

3. **Use `sentinelsat` or Other Libraries:**
   - Use libraries like `sentinelsat` for programmatic access to Earth observation data.

Here’s how you can access MODIS LST data using the NASA Earthdata API and the `pyproj` library:

## JMI Addition: Generate a Token
- https://urs.earthdata.nasa.gov/users/jirvingphd/user_tokens 
    - Use "Authorization: Bearer" header.

To use the NASA Earthdata API with a token, you need to generate a token from the NASA Earthdata Login (EDL) and use it in your API requests.

### Step-by-Step Guide

1. **Generate a Token from NASA Earthdata Login:**
   - Log in to your [NASA Earthdata Login](https://urs.earthdata.nasa.gov).
   - Navigate to the "My Profile" section.
   - Generate a new token under the "User Profile" section.

2. **Use the Token in Your API Requests:**
   - Use the generated token in the `Authorization` header of your API requests.

In [46]:
# where we stored the token locally on our PC
creds_json = "./earthdata_creds.json"
with open(creds_json) as f:
    creds = json.load(f)
print(creds.keys())

# Your NASA Earthdata token
token = creds['token']

dict_keys(['token'])


In [47]:
# Define the search URL and parameters
search_url = 'https://cmr.earthdata.nasa.gov/search/granules.json'
params = {
    'short_name': 'MOD11A2',  # MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid
    'version': '061', 
    'temporal': '2023-06-01T00:00:00Z,2023-08-31T23:59:59Z',  # Desired temporal range
    'bounding_box': '-95.5,29.7,-95.0,30.2',  # Bounding box for Houston urban area
    'page_size': 10,  # Number of results per page
    'page_num': 1
}

# Headers with the token
headers = {
    'Authorization': f'Bearer {token}'
}

# Send the request
response = requests.get(search_url, params=params, headers=headers)

# Check for a successful response
if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print(f"Error: {response.status_code} - {response.text}")

{'feed': {'updated': '2024-07-05T23:03:10.317Z', 'id': 'https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1', 'title': 'ECHO granule metadata', 'entry': [{'producer_granule_id': 'MOD11A2.A2023145.h09v06.061.2023154043201', 'time_start': '2023-05-25T00:00:00.000Z', 'cloud_cover': '0.0', 'updated': '2023-06-02T23:36:49.187Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061', 'data_center': 'LPCLOUD', 'title': 'MOD11A2.A2023145.h09v06.061.2023154043201', 'coordinate_system': 'GEODETIC', 'day_night_flag': 'BOTH', 'time_end': '2023-06-01T23:59:59.000Z', 'id': 'G2704231088-LPCLOUD', 'original_format': 'ECHO10', 'granule_size': '3.23938', 'browse_flag': True, 'polygons': [['20.0041667 -95.7854937 20.0041667 -85.1431699 29.9958333 -92.3808679 29.9958333 -103.9278538 20.0041667 -95.785493

In [48]:
# viewing the data we received from the API
pd.DataFrame(data['feed'])

,updated,id,title,entry
0,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023145.h09v06.061.2023154043201', 'time_start': '2023-05-25T00:00:00.000Z', 'cloud_cover': '0.0', 'updated': '2023-06-02T23:36:49.187Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
1,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023145.h09v05.061.2023154041457', 'time_start': '2023-05-25T00:00:00.000Z', 'cloud_cover': '4.0', 'updated': '2023-06-02T23:20:57.547Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
2,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023153.h09v06.061.2023164032102', 'time_start': '2023-06-02T00:00:00.000Z', 'cloud_cover': '0.0', 'updated': '2023-06-12T23:06:08.832Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
3,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023153.h09v05.061.2023164034529', 'time_start': '2023-06-02T00:00:00.000Z', 'cloud_cover': '3.0', 'updated': '2023-06-12T23:49:49.631Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
4,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023161.h09v06.061.2023170174522', 'time_start': '2023-06-10T00:00:00.000Z', 'cloud_cover': '0.0', 'updated': '2023-06-19T12:57:31.217Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
5,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023161.h09v05.061.2023170174847', 'time_start': '2023-06-10T00:00:00.000Z', 'cloud_cover': '2.0', 'updated': '2023-06-19T13:10:16.444Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
6,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023169.h09v06.061.2023178032028', 'time_start': '2023-06-18T00:00:00.000Z', 'cloud_cover': '0.0', 'updated': '2023-06-26T22:25:12.256Z', 'dataset_id': 'MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Gl..."
7,2024-07-05T23:03:10.317Z,https://cmr.earthdata.nasa.gov:443/search/granules.json?short_name=MOD11A2&version=061&temporal=2023-06-01T00%3A00%3A00Z%2C2023-08-31T23%3A59%3A59Z&bounding_box=-95.5%2C29.7%2C-95.0%2C30.2&page_size=10&page_num=1,ECHO granule metadata,"{'producer_granule_id': 'MOD11A2.A2023169.h09v05.061.2023178033518', 'time_start': '2023-06-18T00:00:00.000Z', 'clo

In [49]:
entries_df = pd.DataFrame(data['feed']['entry'])
entries_df.head()

,producer_granule_id,time_start,cloud_cover,updated,dataset_id,data_center,title,coordinate_system,day_night_flag,time_end,id,original_format,granule_size,browse_flag,polygons,collection_concept_id,online_access_flag,links
0,MOD11A2.A2023145.h09v06.061.2023154043201,2023-05-25T00:00:00.000Z,0.0,2023-06-02T23:36:49.187Z,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,LPCLOUD,MOD11A2.A2023145.h09v06.061.2023154043201,GEODETIC,BOTH,2023-06-01T23:59:59.000Z,G2704231088-LPCLOUD,ECHO10,3.23938,True,[[20.0041667 -95.7854937 20.0041667 -85.1431699 29.9958333 -92.3808679 29.9958333 -103.9278538 20.0041667 -95.7854937]],C2269056084-LPCLOUD,True,"[{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download MOD11A2.A2023145.h09v06.061.2023154043201.hdf', 'hreflang': 'en-US', 'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h09v..."
1,MOD11A2.A2023145.h09v05.061.2023154041457,2023-05-25T00:00:00.000Z,4.0,2023-06-02T23:20:57.547Z,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,LPCLOUD,MOD11A2.A2023145.h09v05.061.2023154041457,GEODETIC,BOTH,2023-06-01T23:59:59.000Z,G2704231280-LPCLOUD,ECHO10,8.02038,True,[[30.0041667 -103.9365814 30.0041667 -92.3886251 39.9958333 -104.4380348 39.9958333 -117.4920864 30.0041667 -103.9365814]],C2269056084-LPCLOUD,True,"[{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download MOD11A2.A2023145.h09v05.061.2023154041457.hdf', 'hreflang': 'en-US', 'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h09v..."
2,MOD11A2.A2023153.h09v06.061.2023164032102,2023-06-02T00:00:00.000Z,0.0,2023-06-12T23:06:08.832Z,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,LPCLOUD,MOD11A2.A2023153.h09v06.061.2023164032102,GEODETIC,BOTH,2023-06-09T23:59:59.000Z,G2709329705-LPCLOUD,ECHO10,3.17643,True,[[20.0041667 -95.7854937 20.0041667 -85.1431699 29.9958333 -92.3808679 29.9958333 -103.9278538 20.0041667 -95.7854937]],C2269056084-LPCLOUD,True,"[{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download MOD11A2.A2023153.h09v06.061.2023164032102.hdf', 'hreflang': 'en-US', 'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h09v..."
3,MOD11A2.A2023153.h09v05.061.2023164034529,2023-06-02T00:00:00.000Z,3.0,2023-06-12T23:49:49.631Z,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,LPCLOUD,MOD11A2.A2023153.h09v05.061.2023164034529,GEODETIC,BOTH,2023-06-09T23:59:59.000Z,G2709350019-LPCLOUD,ECHO10,8.37,True,[[30.0041667 -103.9365814 30.0041667 -92.3886251 39.9958333 -104.4380348 39.9958333 -117.4920864 30.0041667 -103.9365814]],C2269056084-LPCLOUD,True,"[{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download MOD11A2.A2023153.h09v05.061.2023164034529.hdf', 'hreflang': 'en-US', 'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h09v..."
4,MOD11A2.A2023161.h09v06.061.2023170174522,2023-06-10T00:00:00.000Z,0.0,2023-06-19T12:57:31.217Z,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,LPCLOUD,MOD11A2.A2023161.h09v06.061.2023170174522,GEODETIC,BOTH,2023-06-17T23:59:59.000Z,G2715588619-LPCLOUD,ECHO10,3.04064,True,[[20.0041667 -95.7854937 20.0041667 -85.1431699 29.9958333 -92.3808679 29.9958333 -103.9278538 20.0041667 -95.7854937]],C2269056084-LPCLOUD,True,"[{'rel': 'http://esipfed.org/ns/fedsearch/1.1/data#', 'title': 'Download MOD11A2.A2023161.h09v06.061.2023170174522.hdf', 'hreflang': 'en-US', 'href': 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023161.h09v..."


In [50]:
# Extract granule URLs from the search results
granule_urls = [granule['links'][0]['href'] for granule in data['feed']['entry'] if 'links' in granule and granule['links']]
granule_urls

['https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h09v06.061.2023154043201/MOD11A2.A2023145.h09v06.061.2023154043201.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h09v05.061.2023154041457/MOD11A2.A2023145.h09v05.061.2023154041457.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h09v06.061.2023164032102/MOD11A2.A2023153.h09v06.061.2023164032102.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h09v05.061.2023164034529/MOD11A2.A2023153.h09v05.061.2023164034529.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023161.h09v06.061.2023170174522/MOD11A2.A2023161.h09v06.061.2023170174522.hdf',
 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023161.h09v05.061.2023170174847/MOD11A2.A2023161.h09v05.061.2023170174847.hdf',
 'ht

## 🤖Using ChatGPT to Set New Coordinates

1. **Sign up for OpenAI's API:** 
   - Visit the [OpenAI website](https://www.openai.com) and sign up for an API key.

2. **Create a `.secret` folder:**
   ```bash
   cd ~
   mkdir .secret
   ```

3. **Save your API key as a text file in the `.secret` folder:**
   - Open a text editor and paste your API key.
   - Save the file as `open-ai.txt` in the `.secret` folder. For example, you can use the following command in the terminal to create the file and save the API key:
   ```bash
   echo "your_openai_api_key_here" > ~/.secret/open-ai.txt
   ```

4. **Export the key from the file to your `.bash_profile`:**
   - Open your `.bash_profile` for editing:
   ```bash
   code ~/.bash_profile
   ```
   - Add the following line to export the API key:
   ```bash
   export OPENAI_API_KEY=$(cat ~/.secret/open-ai.txt)
   ```
   - Save the file and exit the editor 

5. **Reload your `.bash_profile` to apply the changes:**
   ```bash
   source ~/.bash_profile
   ```

After following these steps, your API key will be available in your environment variables as `OPENAI_API_KEY`.

### Using LangChain with ChatGPT

In [51]:
## Defining the structured output desired from chat gpt
## Tip was to use make_model
# https://stackoverflow.com/questions/63257839/best-way-to-specify-nested-dict-with-pydantic
class Coordinates(BaseModel):
    SW: List[float]
    NE: List[float]
    
class RegionCoordinates(BaseModel):
    rural: Optional[Coordinates]
    urban: Optional[Coordinates]
                            
class DataParams(BaseModel):
    """
    Represents the parameters for data analysis.

    Attributes:
        city_region_name (str): The name of the city or region.
        coordinates (Optional[RegionCoordinates]): The coordinates of the city or region.
        time (Dict[str, str]): A dictionary containing time-related information.
    """
    city_region_name: str
    coordinates: Optional[RegionCoordinates]
    time: Dict[str, str]

### Created a function that asks ChatGPT for suggested parameters

In [52]:
def suggest_data_params(query: str, temperature=0.1, model_type='gpt-4o',
                       return_llm = False, return_json=True) -> str:
    """
    Suggests data parameters for downloading MODIS data for a specific region and time range.
    
    Args:
        query (str): The query describing the requirements for the data download.
        temperature (float, optional): The temperature parameter for the language model. Defaults to 0.1.
        model_type (str, optional): The type of language model to use. Defaults to 'gpt-4o'.
        return_llm (bool, optional): Whether to return the language model chain. Defaults to False.
        return_json (bool, optional): Whether to return the response as JSON. Defaults to True.
    
    Returns:
        str: The response from the language model chain or the JSON response, depending on the value of return_json.
    """
    
    # The prompt template for suggesting data parameters
    prompt = """
    I am performing an urban heat island analysis project with MODIS data comparing urban areas vs. rural areas. 
    I need to download MODIS data for 2 nearby non-overlapping regions (urban area and rural area outside of city) and time range.
    Help me select the urban and rural regions and time following the instructions below.
    {query}
    
    Provide me the data parameters for the download (city_region_name, coordinates as SW [lat,long] NE [lat,long], time_start named 'start', time_end named 'end') in the following format:
    Format Instructions:
    {format_instructions}
    """
    # Create a ChatPromptTemplate object
    final_prompt_template = PromptTemplate.from_template(prompt)

    # Get api key for OpenAI from the environment or session state (if on Streamlit)
    try:
        api_key = st.session_state.OPENAI_API_KEY
    except:
        api_key = os.getenv('OPENAI_API_KEY')
        
    # Instantiate the language model and setting the specific model (chat-gpto is newest and reasonable price)
    # and  set the temperature (creativity level)
    llm = ChatOpenAI(temperature=temperature, model=model_type, api_key=api_key)
    
    if return_json:
        # # JsonOutputParser will use the data model classes from above
        parser = JsonOutputParser(pydantic_object=DataParams,)    
        # Add formatting instructions for pydantic
        instructions =  parser.get_format_instructions()
            
    else:
        ## StrOutputParser will return the response as a string
        parser = StrOutputParser(output_key="response")
        # Manually defining the format instructions
        instructions = "Respond with text for each topic as a nested list with the topic number,  descriptive label,top words, and short insight."
        
    ## Adding the instructions to the prompt template
    final_prompt_template = final_prompt_template.partial(format_instructions=instructions)
    
    
    # Making the final chain
    llm_chain = final_prompt_template | llm | parser
    
    # Return the chain if specified
    if return_llm:
        return llm_chain
    else:
    
        # Invoke the chain with the query to get the response
        response = llm_chain.invoke(input=dict(query=query))
        return response

In [53]:
GET_NEW_LOCATION = False
# Where we are storing our parameters
fpath_params = "./config/data_params.json"

# If we want a new location
if GET_NEW_LOCATION:
    # ask ChatGPT to suggest another set of parameters
    chatgpt_params = suggest_data_params(query="""Select a region in the southern USA to avoid political bias/spin 
                                        and a time range to highlight the effects of climate change (like 06/01/2023-08/31/2023).
                                        Make sure to select a region that does not cover a body of water.
                                        Select small regions from the selected area to minimize the size of the dataset.
                                        Do not use Texas.""", 
                                        return_json=True, temperature=0.0)


else:
    # otherwise, use the parameters we already have
    with open(fpath_params) as f:
        chatgpt_params = json.load(f)

chatgpt_params

{'city_region_name': 'Atlanta, GA',
 'coordinates': {'urban': {'SW': [33.749, -84.388], 'NE': [33.799, -84.338]},
  'rural': {'SW': [33.5, -84.5], 'NE': [33.55, -84.45]}},
 'time': {'start': '2023-06-01', 'end': '2023-08-31'}}

In [54]:
data_params = chatgpt_params
coordinates = data_params['coordinates']
time_range = data_params['time']

In [55]:
## Save params to disk, but check if file exists first and ask user if they want to overwrite
# Save data params
if GET_NEW_LOCATION == True:

    # Check if file exists
    if os.path.exists(fpath_params):
        with open(fpath_params) as f:
            current = json.load(f)

        # ask user if they want to overwrite what is currently stored in the json file    
        ans = input(f"File {fpath_params} exists.\t\n\nParams: {current}\nOverwrite? (y/n)")
        if ans.lower() == 'n':
            print("Not overwriting file.")
        else:
            print(f"Overwriting file with data params: {chatgpt_params}")
            with open(fpath_params, 'w') as f:
                json.dump(chatgpt_params, f)
    else:
        # save parameters to json file
        print(f"Saving data params to {fpath_params}")
        with open(fpath_params, 'w') as f:
            json.dump(chatgpt_params, f)

## Search and Download Data with Pagination and Return List of Entries/Links

Now that we have the parameters for the locations, we need to ask the NASA Earthdata API for the data  

In [56]:
def search_and_download(region_name, bounding_box, time_range, token, dest_folder='./data/MODIS-LST/',
                        force_download=False, verbose=True):
    """
    Searches for granules using the NASA Earthdata API and downloads the data files for a given region.

    Args:
        region_name (str): The name of the region.
        bounding_box (dict): The bounding box coordinates of the region in the format {'SW': [lat, lon], 'NE': [lat, lon]}.
        time_range (dict): The temporal range of the data in the format {'start': 'YYYY-MM-DD', 'end': 'YYYY-MM-DD'}.
        token (str): The access token for the NASA Earthdata API.
        dest_folder (str, optional): The destination folder to save the downloaded data files. Defaults to './data/MODIS-LST/'.

    Returns:
        list: A list of dictionaries containing the region name and the URL of each downloaded data file.
    """
    # Base URL for searching granules
    search_url = 'https://cmr.earthdata.nasa.gov/search/granules.json'
    
    # Pagination settings
    page_size = 10
    page_num = 1
    total_hits = None

    # List to store entries and links
    entries_links = []

    while True:
        # Set up the parameters for the search query
        params = {
            'short_name': 'MOD11A2',  # Dataset short name
            'version': '061',         # Dataset version
            'temporal': f"{time_range['start']},{time_range['end']}",  # Temporal range
            'bounding_box': f"{bounding_box['SW'][1]},{bounding_box['SW'][0]},{bounding_box['NE'][1]},{bounding_box['NE'][0]}",  # Bounding box coordinates
            'page_size': page_size,   # Number of results per page
            'page_num': page_num      # Current page number
        }
        
        # Authorization header with the token
        headers = {
            'Authorization': f'Bearer {token}'
        }

        # Send the request to the NASA Earthdata API
        response = requests.get(search_url, params=params, headers=headers)

        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()


            ## JMI: Confirm this total_hits code works as expected
            # Determine the total number of hits on the first request
            if total_hits is None:
                total_hits = int(response.headers.get('CMR-Hits', 0))
                print(f"Total hits for {region_name}: {total_hits}")

            # Check if there are entries in the response
            if data['feed']['entry']:
                for entry in data['feed']['entry']:
                    # Extract relevant metadata from each entry
                    granule_id = entry.get('id', 'N/A')
                    dataset_id = entry.get('dataset_id', 'N/A')
                    start_time = entry.get('time_start', 'N/A')
                    end_time = entry.get('time_end', 'N/A')
                    spatial_extent = entry.get('boxes', ['N/A'])[0]
                    
                    
                    # Extract the data links for downloading
                    data_links = [link['href'] for link in entry['links'] if 'data#' in link['rel']]
                    
                    # Download each data link and store the entries and links
                    for url in data_links:
                        dir_for_dl = os.path.join(dest_folder, region_name)
                        # Define the filename based on the URL (to check if the file is a directory)
                        filename = os.path.join(dir_for_dl,#dest_folder, 
                                                url.split('/')[-1])
                        
                
                        # Check if directory
                        if os.path.isdir(filename):
                            if verbose:
                                print(f"- Skipping directory {filename}")
                            continue
                        
                        if "s3credentials" in filename:
                            if verbose:
                                print(f"- Skipping S3 credentials link {filename}")
                            continue
                        
                        if '?p' in filename:
                            if verbose:
                                print(f"- Skipping link with query parameters {filename}")
                            continue
                        # Remove question marks
                        filename = filename.replace("?", "-")
                        
                        
                        filepath = download_file(url, dir_for_dl, token, force_download=force_download, 
                                                 verbose=True # Always be verbose for download
                                                 )
                        entries_links.append({'region': region_name, 'url': url,"fpath":filepath, 'granule_id': granule_id, 'dataset_id': dataset_id,
                                            'start_time': start_time, 'end_time': end_time, 'spatial_extent': spatial_extent})
                        # print("\n")
            else:
                print(f"\n[!] No entries found for region: {region_name}")

            # Check if we have fetched all results
            if page_num * page_size >= total_hits:
                break
            else:
                page_num += 1
        else:
            print(f"\n[!] Error: {response.status_code} - {response.text}")
            break

    return entries_links

In [57]:
def download_file(url, dest_folder, token, force_download=False, verbose=True):
    """
    Downloads a file from the given URL and saves it to the specified destination folder.

    Args:
        url (str): The URL of the file to download.
        dest_folder (str): The destination folder where the file will be saved.
        token (str): The authorization token for accessing the file.
        force_download (bool, optional): If set to True, the file will be downloaded even if it already exists in the destination folder. Defaults to False.

    Returns:
        str: The path of the downloaded file.

    Raises:
        None

    """
    # Create the destination folder if it doesn't exist
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    # Define the filename based on the URL
    filename = os.path.join(dest_folder, url.split('/')[-1])
    
    # Check if the file already exists
    if os.path.exists(filename) and not force_download:
        if verbose:
            print(f"- File {filename} already exists, skipping download.")
        return filename

    # Authorization header with the token
    headers = {
        'Authorization': f'Bearer {token}'
    }
    
    try:
        # Send the request to download the file
        response = requests.get(url, headers=headers)
        
    except Exception as e:
        print(f"- [!] An error occurred while downloading {url}: {e}")
        return
    
    # Save the file if the request is successful
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        if verbose:
            print(f"- Downloaded {filename}")
    else:
        print(f"- [!] Failed to download {url}: {response.status_code}")
    
    return filename

In [58]:
## Plot the region suggested
# Function to generate sample points within a bounding box
def generate_sample_points(sw, ne, num_points=10):
    latitudes = [sw[0] + i * (ne[0] - sw[0]) / (num_points - 1) for i in range(num_points)]
    longitudes = [sw[1] + i * (ne[1] - sw[1]) / (num_points - 1) for i in range(num_points)]
    return [(lat, lon) for lat in latitudes for lon in longitudes]
# Dataframe to store results
results = []

# Check if any coordinates within the bounding boxes are over sea
for region, bounds in data_params['coordinates'].items():
    
    sw = bounds['SW']
    ne = bounds['NE']
    
    
    # Generate sample points within the bounding box
    sample_points = generate_sample_points(sw, ne, num_points=10)
    
    for lat, lon in sample_points:
        results.append({'Region': region, 'Latitude': lat, 'Longitude': lon, 'Group': region})

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Plot the results using plotly express
# fig = px.scatter_geo(
#     results_df,
#     lat='Latitude',
#     lon='Longitude',
#     color='Group',
#     symbol='Group',
#     # category_orders={'LandOrSea': ['land', 'sea']},
#     title='Land and Sea Classification of Sample Points within Bounding Boxes',
#     # labels={'LandOrSea': 'Classification'},
#     scope='usa'
# )

fig = px.scatter_mapbox(results_df, lat="Latitude", lon="Longitude", color='Group',
                        # color_continuous_scale="Viridis", 
                        mapbox_style="carto-positron",
                        title="Land and Sea Classification of Sample Points within Bounding Boxes",
                        height=600)

# Add the bounding boxes to the map
for region, bounds in coordinates.items():
    fig.add_trace(
        px.line_geo(
            pd.DataFrame({
                'lat': [bounds['SW'][0], bounds['SW'][0], bounds['NE'][0], bounds['NE'][0], bounds['SW'][0]],
                'lon': [bounds['SW'][1], bounds['NE'][1], bounds['NE'][1], bounds['SW'][1], bounds['SW'][1]]
            }),
            lat='lat',
            lon='lon'
        ).data[0]
    )

# fig.update_geos(
#     visible=False, resolution=50,
#     showcountries=True, countrycolor="Black",
#     showsubunits=True, subunitcolor="Blue"
# )

fig.update_layout(
    height=600,
    # margin={"r":0,"t":0,"l":0,"b":0}
)

fig.show()

In [59]:
# Set DATA_DIR using region name from data params
DATA_DIR = f"./data/MODIS-LST/{data_params['city_region_name'].replace(',','_').replace(' ','')}/"
DATA_DIR

'./data/MODIS-LST/Atlanta_GA/'

This code will search for and download MODIS LST data for the specified area and time period, storing the data files in the specified directory.

In [60]:
# List to store all entries and links
all_entries_links = []

# Iterate through the regions and download data
for region_name, bounding_box in data_params['coordinates'].items():
    time_range = data_params['time']
    # group_links=  []
    entries_links = search_and_download(region_name, bounding_box, time_range, token=creds['token'],
                                        dest_folder=DATA_DIR, force_download=False, 
                                        verbose=False # new verbose flag
                                        )
    all_entries_links.extend(entries_links)
    print('\n\n')

Total hits for urban: 13
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.hdf already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.061 already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.hdf already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.cmr.xml already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.cmr.xml already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.061 already exists, skipping download.
- File ./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2

### Explanation

1. **Define Coordinates and Time Range:**
   - `coordinates`: Dictionary containing bounding box coordinates for each region.
   - `time_range`: Dictionary containing the start and end times for the data search in a flexible format.
   - Use `pd.to_datetime` to convert the flexible date format to the required format (`'%Y-%m-%dT%H:%M:%SZ'`) for the API call.

2. **Search and Download Data with Pagination and Return List of Entries/Links:**
   - **`search_and_download` function:**
     - **Parameters:** `region_name`, `bounding_box`, `time_range`, `dest_folder`, `token`.
     - Constructs the search URL and parameters with the correct time format.
     - Sends the GET request with authorization.
     - Parses the JSON response.
     - Handles pagination to retrieve all pages of results.
     - Downloads data files for each granule entry.
     - Stores the entries and links in a list and returns the list.
   
   - **`download_file` function:**
     - **Parameters:** `url`, `dest_folder`, `token`.
     - Creates the destination folder if it doesn't exist.
     - Downloads the file from the URL with authorization.
     - Saves the file to the specified folder.
   
   - **Main Loop:**
     - Iterates through each region in the `coordinates` dictionary.
     - Calls `search_and_download` for each region to download the data.
     - Collects all entries and links in a list.
   
   - **Save to CSV:**
     - Converts the list of entries and links to a pandas DataFrame.
     - Saves the DataFrame to a CSV file.

By following this approach, you can keep track of which region the data belongs to and save the entries and links to a CSV file for further analysis or reference.

### Saving Files into a Dataframe

In [61]:
# putting files into a dataframe
files_df = pd.DataFrame(all_entries_links)

# Save the data to a CSV file
fpath_all_files_csv = "./data/MODIS-LST/all_files_df.csv"
files_df.to_csv(fpath_all_files_csv,index=False)
pd.read_csv(fpath_all_files_csv).head()

,region,url,fpath,granule_id,dataset_id,start_time,end_time,spatial_extent
0,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.hdf,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.hdf,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,NaN
1,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,NaN
2,urban,s3://lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,NaN
3,urban,https://doi.org/10.5067/MODIS/MOD11A2.061,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.061,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,NaN
4,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h11v05.061.2023164034254/MOD11A2.A2023153.h11v05.061.2023164034254.hdf,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.hdf,G2709346843-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-06-02T00:00:00.000Z,2023-06-09T23:59:59.000Z,NaN


The code below separates the 

In [63]:
# saving both hdf and xml files
hdf_and_xml_files = files_df[files_df['fpath'].str.endswith('.hdf')|files_df['fpath'].str.endswith('.xml')].copy()
# grabbing the end of the filepath and putting in new column
hdf_and_xml_files['type'] = hdf_and_xml_files['fpath'].apply(lambda x: x.split('.')[-1])

# Save to csv
fpath_hdf_and_xml_files_csv = "./data/MODIS-LST/hdf_and_xml_files.csv"
hdf_and_xml_files.to_csv(fpath_hdf_and_xml_files_csv, index=False)
hdf_and_xml_files.head()

,region,url,fpath,granule_id,dataset_id,start_time,end_time,spatial_extent,type
0,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.hdf,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.hdf,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,N/A,hdf
1,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,N/A,xml
2,urban,s3://lp-prod-protected/MOD11A2.061/MOD11A2.A2023145.h11v05.061.2023154043129/MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023145.h11v05.061.2023154043129.cmr.xml,G2704231484-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-05-25T00:00:00.000Z,2023-06-01T23:59:59.000Z,N/A,xml
4,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h11v05.061.2023164034254/MOD11A2.A2023153.h11v05.061.2023164034254.hdf,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.hdf,G2709346843-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-06-02T00:00:00.000Z,2023-06-09T23:59:59.000Z,N/A,hdf
5,urban,https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/MOD11A2.061/MOD11A2.A2023153.h11v05.061.2023164034254/MOD11A2.A2023153.h11v05.061.2023164034254.cmr.xml,./data/MODIS-LST/Atlanta_GA/urban\MOD11A2.A2023153.h11v05.061.2023164034254.cmr.xml,G2709346843-LPCLOUD,MODIS/Terra Land Surface Temperature/Emissivity 8-Day L3 Global 1km SIN Grid V061,2023-06-02T00:00:00.000Z,2023-06-09T23:59:59.000Z,N/A,xml


In [64]:
# storing hdf and xml files in their own dataset
xml_files = hdf_and_xml_files.loc[hdf_and_xml_files['type'] == 'xml'].drop(columns=['url'])
hdf_files = hdf_and_xml_files.loc[hdf_and_xml_files['type'] == 'hdf'].drop(columns=['url'])

### Using Xarray on the first row of the dataset

In [65]:
# grabbing one ropw of the dataset to test on
hdf_fpath = hdf_files.iloc[0]['fpath']

# Load in example dataset
dataset = xr.open_dataset(hdf_fpath, engine='netcdf4')
[i for i in dir(dataset) if not i.startswith('_')]
# dataset
print(dataset)

<xarray.Dataset>
Dimensions:           (YDim:MODIS_Grid_8Day_1km_LST: 1200,
                       XDim:MODIS_Grid_8Day_1km_LST: 1200)
Dimensions without coordinates: YDim:MODIS_Grid_8Day_1km_LST,
                                XDim:MODIS_Grid_8Day_1km_LST
Data variables:
    LST_Day_1km       (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Day            (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_time     (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_angl     (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    LST_Night_1km     (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Night          (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Night_view_time   (YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Night_view_angl   (YDim:MODIS_Grid_8Day_1km_LST, XDim:MOD

### Checking to make sure Coordinates are on land

To determine whether your coordinates are over land or sea, you can use a geographic information system (GIS) library like `geopandas` along with a shapefile that contains land and sea boundaries. One such dataset is the Natural Earth dataset, which provides vector data for land and water boundaries.

Here’s how you can achieve this using Python:

1. **Install the required libraries:**
   - `geopandas`: For handling geographic data.
   - `shapely`: For geometric operations.

2. **Download the Natural Earth dataset:**
   - You can download the Natural Earth land polygons dataset from [Natural Earth](https://www.naturalearthdata.com/downloads/110m-physical-vectors/).

3. **Load the shapefile and check the coordinates:**

In [66]:
# Load the Natural Earth land polygons shapefile (see above link to download data)
land_shapefile = "./data/110m_physical/ne_110m_land.shp" #"path/to/ne_110m_land.shp"
land = gpd.read_file(land_shapefile, engine='fiona')
land

,featurecla,scalerank,min_zoom,geometry
0,Land,1,1.0,"POLYGON ((-59.57209 -80.04018, -59.86585 -80.54966, -60.15966 -81.00033, -62.25539 -80.86318, -64.48813 -80.92193, -65.74167 -80.58883, -65.74167 -80.54966, -66.29003 -80.25577, -64.03769 -80.29494, -61.88325 -80.39287, -61.13898 -79.98137, -60.6..."
1,Land,1,1.0,"POLYGON ((-159.20818 -79.49706, -161.1276 -79.63421, -162.43985 -79.28147, -163.02741 -78.92877, -163.0666 -78.86997, -163.7129 -78.59567, -163.7129 -78.59567, -163.1058 -78.22334, -161.24511 -78.38018, -160.24621 -78.69365, -159.4824 -79.04634, ..."
2,Land,1,0.0,"POLYGON ((-45.15476 -78.04707, -43.92083 -78.4781, -43.48995 -79.08556, -43.37244 -79.51664, -43.33327 -80.02612, -44.88054 -80.33964, -46.50617 -80.59436, -48.38642 -80.82948, -50.48211 -81.02544, -52.85199 -80.96669, -54.16426 -80.63353, -53.98..."
3,Land,1,1.0,"POLYGON ((-121.21151 -73.50099, -119.91885 -73.65773, -118.72414 -73.48135, -119.29212 -73.8341, -120.23222 -74.08881, -121.62283 -74.01047, -122.62173 -73.65778, -122.62174 -73.65778, -122.40624 -73.32462, -121.21151 -73.50099))"
4,Land,1,1.0,"POLYGON ((-125.55957 -73.48135, -124.03188 -73.87327, -124.61947 -73.8341, -125.91218 -73.73612, -127.28313 -73.46177, -127.28313 -73.46177, -126.55847 -73.24623, -125.55957 -73.48135))"
...,...,...,...,...
122,Land,1,1.0,"POLYGON ((51.13619 80.54728, 49.79368 80.41543, 48.89441 80.33957, 48.75494 80.17547, 47.58612 80.01018, 46.50283 80.24725, 47.07246 80.55942, 44.84696 80.58981, 46.79914 80.77192, 48.31848 80.78401, 48.52281 80.51457, 49.09719 80.75399, 50.03977..."
123,Land,0,0.0,"POLYGON ((99.93976 78.88094, 97.75794 78.7562, 94.97259 79.04475, 93.31288 79.4265, 92.5454 80.14379, 91.18107 80.34146, 93.77766 81.0246, 95.9409 81.2504, 97.88385 80.74698, 100.18666 79.78014, 99.93976 78.88094))"
124,Land,0,0.0,"POLYGON ((-87.02 79.66, -85.81435 79.3369, -87.18756 79.0393, -89.03535 78.28723, -90.80436 78.21533, -92.87669 78.34333, -93.95116 78.75099, -93.93574 79.11373, -93.14524 79.3801, -94.974 79.37248, -96.07614 79.70502, -96.70972 80.15777, -96.016..."
125,Land,0,0.0,"POLYGON ((-68.5 83.10632, -65.82735 83.02801, -63.68 82.9, -61.85 82.6286, -61.89388 82.36165, -64.334 81.92775, -66.75342 81.72527, -67.65755 81.50141, -65.48031 81.50657, -67.84 80.9, -69.4697 80.61683, -71.18 79.8, -73.2428 79.63415, -73.88 79..."


In [67]:
# Function to check if a coordinate is over land
def is_land(lat, lon, land_gdf):
    point = Point(lon, lat)
    return any(land_gdf.contains(point))

In [68]:

# Function to generate sample points within a bounding box
def generate_sample_points(sw, ne, num_points=10):
    latitudes = [sw[0] + i * (ne[0] - sw[0]) / (num_points - 1) for i in range(num_points)]
    longitudes = [sw[1] + i * (ne[1] - sw[1]) / (num_points - 1) for i in range(num_points)]
    return [(lat, lon) for lat in latitudes for lon in longitudes]

Looping through all the regions and bounds within our coordinates to identify how many points are over land and sea (should all be over land)

In [69]:
# # Define your coordinates
# coordinates = {
#     'urban': {"SW": [29.5, -95.5], "NE": [30.0, -95.0]},
#     'rural': {"SW": [30.5, -96.5], "NE": [31.0, -96.0]},
# }

# Check if any coordinates within the bounding boxes are over sea
# coordinates variable comes from: data_params['coordinates']
for region, bounds in coordinates.items():
    sw = bounds['SW']
    ne = bounds['NE']
    
    # Generate sample points within the bounding box
    sample_points = generate_sample_points(sw, ne, num_points=100)
    
    sea_points = []
    land_points = []
    for lat, lon in sample_points:
        if is_land(lat, lon, land):
            land_points.append((lat, lon))
        else:
            sea_points.append((lat, lon))
    
    if sea_points:
        print(f"Region: {region} has points over the sea.")
    else:
        print(f"Region: {region} is entirely over land.")
    
    
    # Print details
    print(f"Number of land points: {len(land_points)}")
    print(f"Number of sea points: {len(sea_points)}")

Region: urban is entirely over land.
Number of land points: 10000
Number of sea points: 0
Region: rural is entirely over land.
Number of land points: 10000
Number of sea points: 0


In [71]:
file_list_dict = hdf_files.groupby('region')['fpath'].apply(list).to_dict()
fpath = file_list_dict['rural'][0]
fpath

'./data/MODIS-LST/Atlanta_GA/rural\\MOD11A2.A2023145.h10v05.061.2023154043348.hdf'

In [74]:
USE_DICT= True


shape_list = []

if USE_DICT:
    # Loop to load all files
    dataset_dict = {'urban':[],
                    'rural':[]}
else:
    dataset_list = []

for group, file_list in file_list_dict.items():
    for fpath in file_list:
        # Open the HDF5 file
    
        dataset = xr.open_dataset(fpath, engine='netcdf4')
    
        # Save the group as an attribute
        dataset = dataset.expand_dims(group=[group])
        
        shape_list.append([group, dataset['LST_Day_1km'].shape])
        # dataset.attrs['group'] = group
        
        
        
        if USE_DICT:
            dataset_dict[group].append(dataset)
        else:
            dataset_list.append(dataset)
        

if USE_DICT:
    print(f'- Using dictionary')
    # Combine all the datasets into a single dataset
    ds_rural = xr.concat(dataset_dict['rural'], dim='time')
    ds_urban = xr.concat(dataset_dict['urban'], dim='time')
    
    # Combined dataset (added back)
    ds = xr.concat([ds_rural, ds_urban], dim='time')

else:
    print(f'- Using list')
    ds = xr.concat(dataset_list, dim='time')

print(ds)

- Using dictionary


c:\Users\Chris\anaconda3\envs\nasa-env\Lib\site-packages\xarray\core\concat.py:544: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.

c:\Users\Chris\anaconda3\envs\nasa-env\Lib\site-packages\xarray\core\concat.py:544: FutureWarning:

unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.



<xarray.Dataset>
Dimensions:           (group: 2, time: 26, YDim:MODIS_Grid_8Day_1km_LST: 1200,
                       XDim:MODIS_Grid_8Day_1km_LST: 1200)
Coordinates:
  * group             (group) object 'rural' 'urban'
Dimensions without coordinates: time, YDim:MODIS_Grid_8Day_1km_LST,
                                XDim:MODIS_Grid_8Day_1km_LST
Data variables:
    LST_Day_1km       (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Day            (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_time     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_angl     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    LST_Night_1km     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Night          (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST

In [76]:
ds_urban

<xarray.Dataset>
Dimensions:           (group: 1, time: 13, YDim:MODIS_Grid_8Day_1km_LST: 1200,
                       XDim:MODIS_Grid_8Day_1km_LST: 1200)
Coordinates:
  * group             (group) object 'urban'
Dimensions without coordinates: time, YDim:MODIS_Grid_8Day_1km_LST,
                                XDim:MODIS_Grid_8Day_1km_LST
Data variables:
    LST_Day_1km       (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Day            (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_time     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Day_view_angl     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    LST_Night_1km     (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    QC_Night          (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Night_view_time   (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Night_view_angl   (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Emis_31           (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Emis_32           (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Clear_sky_days    (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
    Clear_sky_nights  (time, group, YDim:MODIS_Grid_8Day_1km_LST, XDim:MODIS_Grid_8Day_1km_LST) float32 ...
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    identifier_product_doi:            10.5067/MODIS/MOD11A2.061
    identifier_product_doi_authority:  https://doi.org
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...

In [77]:
i_sel = -1
df_rural = ds_rural['LST_Day_1km'].isel(time=i_sel).to_dataframe().reset_index()
df_urban = ds_urban['LST_Day_1km'].isel(time=i_sel).to_dataframe().reset_index()

In [80]:
df_urban

,group,YDim:MODIS_Grid_8Day_1km_LST,XDim:MODIS_Grid_8Day_1km_LST,LST_Day_1km
0,urban,0,0,301.279999
1,urban,0,1,301.160004
2,urban,0,2,301.100006
3,urban,0,3,300.880005
4,urban,0,4,300.779999
...,...,...,...,...
1439995,urban,1199,1195,NaN
1439996,urban,1199,1196,NaN
1439997,urban,1199,1197,NaN
1439998,urban,1199,1198,NaN
